In [19]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827 cell culture'


# Get info

In [22]:
markers_map = {
    'cycle1': {
        1: 'Hoeschst', 
        2: 'TEAD1 & YAP1'
    },
    'cycle2': {
        1: 'Hoeschst', 
        2: 'CylinE & CDK2'
    },
    'cycle3': {
        1: 'Hoeschst', 
        2: 'P-ERK & c-MYC'
    },
    'cycle4': {
        1: 'Hoeschst', 
        2: 'p-AKT & mTOR'
    },
    'cycle5': {
        1: 'Hoeschst', 
        2: 'Mcl-1 & BAK'
    },
    'cycle6': {
        1: 'Hoeschst',
        2: 'p-EGFR',
        3: 'Phalloidin',
        4: 'Tom20'
    },
    'cycle7': {
        1: 'Hoeschst',
        2: 'Pan-cytokeratin',
        3: 'Golph4',
        4: 'Ki67'
    },
    'cycle8': {
        1: 'Hoeschst',
        2: 'NBD-C6',
        4: 'COX IV'
    },
    'cycle9': {
        1: 'Hoeschst',
        2: 'Concanavalin A',
        4: 'WGA'
    },
}

def get_info(data_raw, marker_dict = markers_map):
    timepoints = []
    fovs = []
    cycles = []
    afs = []
    channels = []
    markers = []
    rois = []
    z_stacks = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name and "stitched" not in name \
            and 'Overlay' not in name \
            and 'Composite' not in name \
            and 'defocused' not in dirpath:
                # Get information from image name
                d_split = dirpath.split('\\')
                n_split = name.split('_')
                                
                time = d_split[-1].split('_')[0]
                fov = d_split[-1].split('_')[-1]
                
                ch = int(n_split[3][2])
                roi = int(n_split[1])
                z = int(n_split[2][1:])
                
                try:
                    cycle = d_split[-1].split('_')[1]
                    if 'Af' in cycle:
                        after_bleach = True
                    else:
                        after_bleach = False
                    cycle = f'cycle{cycle[-1]}'
                    marker = marker_dict[cycle][ch]
                except:
                    continue
        
                timepoints.append(time)
                fovs.append(fov)
                cycles.append(cycle)
                afs.append(after_bleach)
                channels.append(ch)
                markers.append(marker)
                rois.append(roi)
                z_stacks.append(z)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Timepoint": timepoints,
            "FOV": fovs,
            "Cycle": cycles,
            "AfBleach": afs,
            "Channels": channels,
            "Markers": markers,
            "ROI": rois,
            "Z": z_stacks,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [23]:
df_meta_path = data_dir / 'OCT Cell Culture' /'3D' / 'metadata' / 'info.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw)
    df.loc[df.Timepoint == 'HCC827Osimj', 'Timepoint'] = 'HCC827Osim'
    df.loc[df.Timepoint == 'HCC827ctrl', 'Timepoint'] = 'HCC827Ctrl'

    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Loaded df


In [24]:
df = df[df.AfBleach==False]

In [25]:
df.head()

,Timepoint,FOV,Cycle,AfBleach,Channels,Markers,ROI,Z,Path
0,HCC827Osim,FW1,cycle3,False,1,Hoeschst,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
1,HCC827Osim,FW1,cycle3,False,2,P-ERK & c-MYC,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
2,HCC827Osim,FW1,cycle3,False,1,Hoeschst,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
3,HCC827Osim,FW1,cycle3,False,2,P-ERK & c-MYC,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
4,HCC827Osim,FW1,cycle3,False,1,Hoeschst,1,3,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."


In [26]:
df.Timepoint.unique()

array(['HCC827Osim', 'HCC827Ctrl'], dtype=object)

# Save to hdf5

In [27]:
import h5py
import multiprocessing
import functools 

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        chunk_shape = (1,) + data_shape[1:]
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=chunk_shape,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path)

def joblib_loop(task, pics):
    return Parallel(n_jobs=multiprocessing.cpu_count()//2)(delayed(task)(i) for i in pics)

def save_imgs(df_channel, n, file_path):
    marker = df_channel.iloc[0].Markers
    paths = df_channel.Path.to_numpy()

    imgs = joblib_loop(read_img, paths)
    imgs = np.array(imgs)
    info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}

    # hdf5 as Channel -> Z mapping
    save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)

In [28]:
df_imgs_path = data_dir / 'OCT Cell Culture' / '3D' / 'metadata' / 'imgs.csv'

try:
    df_imgs_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
    
temp_path = data_dir / 'OCT Cell Culture' / '3D' / 'hdf5' / 'ROI_raw'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Timepoint', 'FOV', 'AfBleach', 'ROI']) 
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        print(name)
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])

        group_channel = df_group.groupby(['Cycle', 'Channels'])
        for n, df_channel in group_channel:
            marker = df_channel.iloc[0].Markers
            paths = df_channel.Path.to_numpy()

            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Timepoint', 'FOV', 'AfBleach', 'ROI', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Folder is already there
Loaded df


# Save tiffile

In [29]:
import tifffile as tiff

# Read imgs and blur info
df_imgs_path = data_dir / 'OCT Cell Culture' / '3D' / 'metadata' / 'imgs.csv'
df_imgs = pd.read_csv(df_imgs_path)

save_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'imgs' / 'raw'
save_path.mkdir(parents=True, exist_ok=True)

In [30]:
group = df_imgs.groupby(['Timepoint', 'FOV'])
for name, df_group in group:
    
    path = df_group.iloc[0].Path
    for i, row in tqdm(enumerate(df_group.itertuples()), total=len(df_group)):
        path = row.Path

        # Read images
        cycles = []
        imgs_all = []
        channels = []
        with h5py.File(path, "r") as f:
            for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
                cycle = k.split('_')[0]
                channel = f[k].attrs['Channel']

                imgs = f[k][:]
                cycles.append(cycle)
                channels.append(channel)
                imgs_all.append(imgs)
                       
        cycles = np.array(cycles)
        channels = np.array(channels)

        # Get imgs_stacked
        z_min = np.min([i.shape[0] for i in imgs_all])
        imgs_stacked = np.stack([i[:z_min] for i in imgs_all])
        for cycle in np.unique(cycles):
            indices = np.where(cycles == cycle)[0]
            imgs = imgs_stacked[indices,...]

            for z in range(imgs.shape[1]):
                 # Create temp path
                name = [row[1], row[2], row[3], z]
                temp_path =  save_path / '_'.join(np.array(name).astype(str))
                temp_path.mkdir(parents=True, exist_ok=True)

                file_name = '_'.join(np.array(["{:03d}".format(row[4]), cycle]).astype(str)) + '.tif'
                file_path = temp_path / file_name

                if os.path.exists(file_path):
                    continue
                
                # Write image
                tiff.imwrite(file_path, imgs[:,z,...])

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [31]:
# This is another version that match best focus z for each ROI

# group = df_imgs.groupby(['Timepoint', 'FOV'])
# for name, df_group in group:
#     df_query = df_blur.query('Timepoint==@name[0] & FOV==@name[1]')
#     path = df_group.iloc[0].Path
    
#     # Get best focus matching
#     matched, left, right = get_range_blur(df_query, path)
#     left_min = np.min(left)
#     right_min = np.min(right)
    
#     assert len(matched) == len(df_query)
#     for i, row in tqdm(enumerate(df_query.itertuples()), total=len(df_query)):
#         path = row.Path
#         z_left = matched[i]+1 - left_min
#         z_right = matched[i]+1 + right_min
        
#         # Read images
#         cycles = []
#         imgs_all = []
#         channels = []
#         with h5py.File(path, "r") as f:
#             for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
#                 cycle = k.split('_')[0]
#                 channel = f[k].attrs['Channel']
#                 if row.Cycle in cycle:

#                     imgs = f[k][z_left:z_right]
#                     cycles.append(cycle)
#                     channels.append(channel)
#                     imgs_all.append(imgs)
                       
#         cycles = np.array(cycles)
#         channels = np.array(channels)

#         # Get imgs_stacked
#         imgs_stacked = np.stack(imgs_all)
#         for cycle in np.unique(cycles):
#             indices = np.where(cycles == cycle)[0]
#             imgs = imgs_stacked[indices,...]

#             for z in range(imgs.shape[1]):
#                  # Create temp path
#                 name = [row[1], row[2], row[3], z]
#                 temp_path = data_dir / 'OCT Cell Culture' / '3D' / 'imgs_to_stitch' / '_'.join(np.array(name).astype(str))
#                 temp_path.mkdir(parents=True, exist_ok=True)

#                 file_name = '_'.join(np.array(["{:03d}".format(row[4]), cycle]).astype(str)) + '.tif'
#                 file_path = temp_path / file_name

#                 if os.path.exists(file_path):
#                     continue
                
#                 # Write image
#                 tiff.imwrite(file_path, imgs[:,z,...])

# Registration

In [33]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

In [34]:
# Loop all images
thumb_dir = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'thumbnails'
imgs_dir = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'imgs' / 'raw'
save_dir = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'registered'


imgs_dir_list = os.listdir(imgs_dir)

thumb_dir.mkdir(parents=True, exist_ok=True)
save_dir.mkdir(parents=True, exist_ok=True)

for dir_path in tqdm(imgs_dir_list):
    thumb_path = thumb_dir / (dir_path+'.tiff')
    if thumb_path.exists():
        continue
    # if 'HCC827Ctrl_FW2' not in dir_path:
    #     continue
        
    # Create reader for each cycle
    readers = []
    for i in range(1, 10):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_cycle'+f'{i}.tif',
            overlap=0.29,
            width=6,
            height=3,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
        
    # Create thumbnails
    reader_1 = readers[0]
    process_axis_flip(reader_1, flip_x=False, flip_y=False)
    thumbnail_1 = thumbnail.make_thumbnail(reader_1, channel=0, scale=0.5)
    tiff.imwrite(thumb_path, thumbnail_1)
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, 9):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()
    

  0%|          | 0/107 [00:00<?, ?it/s]

    assembling thumbnail 18/18
    assembling thumbnail 18/18


\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\utils.py:23: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = skimage.registration.phase_cross_correlation(


    assembling thumbnail 18/18
    estimated cycle offset [y x] = [14. 40.]


\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\thumbnail.py:35: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = phase_cross_correlation(
\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\utils.py:23: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = skimage.registration.phase_cross_correlation(


...

\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\thumbnail.py:35: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = phase_cross_correlation(
\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\utils.py:23: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = skimage.registration.phase_cross_correlation(


    assembling thumbnail 18/18
    estimated cycle offset [y x] = [-16.        51.999996]


\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\thumbnail.py:35: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = phase_cross_correlation(
\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\utils.py:23: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = skimage.registration.phase_cross_correlation(


    assembling thumbnail 18/18
    estimated cycle offset [y x] = [-2. 36.]


\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\thumbnail.py:35: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = phase_cross_correlation(
\\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\thomas\15_pla\src\ashlar-master\ashlar\utils.py:23: FutureWarning: In scikit-image 0.21, phase_cross_correlation will start returning a tuple or 3 items (shift, error, phasediff) always. To enable the new return behavior and silence this warning, use return_error='always'.
  shift = skimage.registration.phase_cross_correlation(


# Save Data

In [36]:
markers_map = {
    'cycle1': {
        'ch0': 'Hoeschst', 
        'ch1': 'TEAD1 & YAP1'
    },
    'cycle2': {
        'ch0': 'Hoeschst', 
        'ch1': 'CylinE & CDK2'
    },
    'cycle3': {
        'ch0': 'Hoeschst', 
        'ch1': 'P-ERK & c-MYC'
    },
    'cycle4': {
        'ch0': 'Hoeschst', 
        'ch1': 'p-AKT & mTOR'
    },
    'cycle5': {
        'ch0': 'Hoeschst', 
        'ch1': 'Mcl-1 & BAK'
    },
    'cycle6': {
        'ch0': 'Hoeschst',
        'ch1': 'p-EGFR',
        'ch2': 'Phalloidin',
        'ch3': 'Tom20'
    },
    'cycle7': {
        'ch0': 'Hoeschst',
        'ch1': 'Pan-cytokeratin',
        'ch2': 'Golph4',
        'ch3': 'Ki67'
    },
    'cycle8': {
        'ch0': 'Hoeschst',
        'ch1': 'NBD-C6',
        'ch2': 'COX IV'
    },
    'cycle9': {
        'ch0': 'Hoeschst',
        'ch1': 'Concanavalin A',
        'ch2': 'WGA'
    },
}

def get_info(data_raw, marker_dict = markers_map):
    timepoints = []
    fovs = []
    Zs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                                
                time=n_split[0]
                fov=n_split[1]
                z=n_split[3]
                cycle=n_split[4]
                ch = n_split[5][:3]
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                    
                timepoints.append(time)
                fovs.append(fov)
                Zs.append(z)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Timepoint": timepoints,
            "FOV": fovs,
            "Z": Zs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df


def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [37]:
data_raw = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'registered'
df_meta_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


In [38]:
df.head()

,Timepoint,FOV,Z,Cycle,Channels,Markers,Path
0,HCC827Ctrl,FW1,0,cycle1,ch0,Hoeschst,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
1,HCC827Ctrl,FW1,0,cycle1,ch1,TEAD1 & YAP1,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
2,HCC827Ctrl,FW1,0,cycle2,ch0,Hoeschst,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
3,HCC827Ctrl,FW1,0,cycle2,ch1,CylinE & CDK2,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
4,HCC827Ctrl,FW1,0,cycle3,ch0,Hoeschst,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...


In [39]:
df_imgs_path = data_dir / 'OCT Cell Culture' / '3D_Whole' /  'metadata' / 'imgs_sti.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)
    
temp_path = data_dir / 'OCT Cell Culture' / '3D_Whole' /  'hdf5' / 'registered'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Timepoint','FOV', 'Z'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        if file_path.exists():
            continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = get_min(imgs)
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Timepoint', 'FOV', 'Z', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/107 [00:00<?, ?it/s]

# Show

In [43]:
df_imgs

,Timepoint,FOV,Z,Path
0,HCC827Ctrl,FW1,0,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
1,HCC827Ctrl,FW1,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
2,HCC827Ctrl,FW1,10,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
3,HCC827Ctrl,FW1,11,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
4,HCC827Ctrl,FW1,12,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
...,...,...,...,...
102,HCC827Osim,FW2,5,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
103,HCC827Osim,FW2,6,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
104,HCC827Osim,FW2,7,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...
105,HCC827Osim,FW2,8,Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Cult...


In [44]:
path = df_imgs.iloc[2].Path

In [45]:
with h5py.File(path, "r") as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']

In [50]:
import napari

napari.view_image(imgs, channel_axis=0, name=markers, visible=False, contrast_limits=[0,2**16])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1726.5, 4308.5), zoom=0.0893611433433898, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1, 1), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3454.0, 1.0), (0.0, 8618.0, 1.0)), current_step=(1727, 4309), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Hoeschst' at 0x2542abca190>, <Image layer 'TEAD1 & YAP1' at 0x2542abce490>, <Image layer 'Hoeschst [1]' at 0x2542b7af8b0>, <Image layer 'CylinE & CDK2' at 0x254432f9d90>, <Image layer 'Hoeschst [2]' at 0x254433bc2b0>, <Image layer 'P-ERK & c-MYC' at 0x254438f8790>, <Image layer 'Hoeschst [3]' at 0x254439f2ca0>, <Image layer 'p-AKT & mTOR' at 0x25443a161c0>, <Image layer 'Hoeschst [4]' at 0x25443b116a0>, <Image layer 'Mcl-1 & BAK' at 0x25443bb1b80>

# Create segmentation training set

In [54]:
import tensorflow as tf 
from skimage import exposure, util

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def contrast_str(img, n_min=0.01, n_max=99.95):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [55]:
cyto_markers = ['p-EGFR', 'Pan-cytokeratin']
# cyto_markers = ['p-EGFR']

In [57]:
whole_seg_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'imgs' / 'segmentation'
crop_seg_path =  data_dir / 'OCT Cell Culture' / '3D_Whole' / 'imgs' / 'training_seg'

whole_seg_path .mkdir(parents=True, exist_ok=True)
crop_seg_path .mkdir(parents=True, exist_ok=True)

N_crop = 1

for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']
    
    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi = imgs[0]
    imgs_cyto = imgs[indices,:]
    
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi)
    # img_cyto = contrast_str(imgs_cyto[0], n_max=99)
    imgs_cyto_scaled = [contrast_str(imgs_cyto[0],n_max=99.5), contrast_str(imgs_cyto[1])]
    img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:4])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)
    
    for i in range(N_crop):
        img_cropped = random_crop(img_rgb, 1000, 1000).numpy().astype(np.uint8)
        
        file_name = f'{"_".join(row[1:4])}_{i}.tif'
        file_path = crop_seg_path / file_name
        tiff.imwrite(file_path, img_cropped)
